In [0]:
import time

In [19]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### 입력, 출력
 * 28 * 28 * 1 
 * 마지막 1은 특징의 개수이다. (MNIST 데이터는 회색조 이미지라 채널에 색상이 하나)
 * 출력값 10개 분류
 * 드롭아웃을 위한 keep_prob 플레이스 홀더로 정의.

In [0]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

### CNN계층 구성

In [0]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)

* X와 첫 번째 계층의 가중치 W1을 가진다.
* 오른쪽과 아래쪽으로 한 칸씩 움직이는 32개의 커널을 가진 컨볼루션 계층을 만들겠다.
* padding = "SAME"은 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직이는 옵션이다.
* tf.nn.relu(L1) : 활성화 함수 'relu'로 지정

In [0]:
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                   padding = 'SAME')

* 앞에서 만든 컨볼루션 계층을 입력층으로 사용 후, 커널 크기를 2 X 2로 하는 풀링 계층을 만든다.
* strides=[1,2,2,1]값은 슬라이딩 시 두 칸씩 움직이겠다는 옵션

### 두번째 층을 만들기 W2, L2

In [0]:
# [3,3,32,64] 32는 컨볼루션 계층의 커널 수
# 이것은 또한 출력층의 개수, 첫번째 컨볼루션이 찾아낸 이미지의 특징개수 
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],
                   padding='SAME')


* 7 X 7 X 64 
* 7 X 7 X 64를 tf.reshape를 이용하여 1차원 계층으로 만든다
* 이 배열 전체를 최종 출력값의 중간 단계인 256개 뉴런으로 연결하는 신경망을 만들어준다.
* 인접한 계측의 모든 뉴런과 상호 연결된 계층을 완전 연결 계측

In [0]:
## W3
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7* 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [0]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3 , W4)

### 손실함수와 AdamOptimizer를 이용한 최적의 함수를 만든다.

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2( 
                      logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
optimizer1 = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [14]:
start = time.time()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size=100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], 
                              feed_dict={X:batch_xs, 
                                         Y:batch_ys, 
                                         keep_prob:0.7})
        total_cost += cost_val
    print("Epoch: %04d" % (epoch + 1), 
          "AVG. cost = {:.3f}".format(total_cost/total_batch))
print("최적화 완료!")

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

end = time.time() - start

Epoch: 0001 AVG. cost = 0.364
Epoch: 0002 AVG. cost = 0.115
Epoch: 0003 AVG. cost = 0.084
Epoch: 0004 AVG. cost = 0.065
Epoch: 0005 AVG. cost = 0.055
Epoch: 0006 AVG. cost = 0.046
Epoch: 0007 AVG. cost = 0.040
Epoch: 0008 AVG. cost = 0.035
Epoch: 0009 AVG. cost = 0.031
Epoch: 0010 AVG. cost = 0.028
Epoch: 0011 AVG. cost = 0.023
Epoch: 0012 AVG. cost = 0.022
Epoch: 0013 AVG. cost = 0.021
Epoch: 0014 AVG. cost = 0.017
Epoch: 0015 AVG. cost = 0.018
최적화 완료!


In [16]:
print("소요 시간 : ", end)
###########
# 결과 확인
###########
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도 :", sess.run(accuracy, 
                           feed_dict={X:mnist.test.images.reshape(
                           -1, 28, 28, 1),
                                     Y:mnist.test.labels,
                                     keep_prob:False}))

소요 시간 :  906.3740820884705
걸린시간 :  906.3740820884705
정확도 : 0.098


## 고수준 API 사용하기 

In [0]:
'''
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                   padding='SAME')

-->
L1 = tf.layers.conv2d(X, 32, [3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])

W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7* 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

--> 
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
'''
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.float32)

L1 = tf.layers.conv2d(X, 32, [3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(X, 64, [3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model,
                                                             labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)




In [21]:
start = time.time()

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size=100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], 
                              feed_dict={X:batch_xs, 
                                         Y:batch_ys, 
                                         is_training:0.7})
        total_cost += cost_val
    print("Epoch: %04d" % (epoch + 1), 
          "AVG. cost = {:.3f}".format(total_cost/total_batch))
print("최적화 완료!")

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

end = time.time() - start

###########
# 결과 확인
###########
print("걸린시간 : ", end)
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도 :", sess.run(accuracy, 
                           feed_dict={X:mnist.test.images.reshape(
                           -1, 28, 28, 1),
                                     Y:mnist.test.labels,
                                     is_training:False}))

KeyboardInterrupt: ignored